<a href="https://colab.research.google.com/github/valintea/Trabajo_Tecnicas/blob/main/Trabajo_T%C3%A9cnicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv('Tweets.csv')



In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [12]:
for tweet in df['text'][90:120]:
    print(tweet)

@VirginAmerica "You down with RNP?" "Yeah you know me!"
@VirginAmerica hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
@VirginAmerica I like the TV and interesting video . Just disappointed in Cancelled Flightled flight when other flights went out to jfk on Saturday .
@VirginAmerica just landed in LAX, an hour after I should of been here. Your no Late Flight bag check is not business travel friendly #nomorevirgin
@VirginAmerica why is flight 345 redirected?
@VirginAmerica Is it me, or is your website down?  BTW, your new website isn't a great user experience.  Time for another redesign.
@VirginAmerica I can't check in or add a bag. Your website isn't working. I've tried both desktop and mobile http://t.co/AvyqdMpi1Y
@VirginAmerica - Let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
@virginamerica What is your phone number. I can't find who to ca

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Supongamos que tu DataFrame se llama df y tiene las columnas 'texto' y 'sentimiento_aerolinea'

# Crear un vectorizador para contar las palabras en los textos
vectorizador = CountVectorizer()

# Aplicar el vectorizador a todo el conjunto de datos
X = vectorizador.fit_transform(df['text'])

# Obtener las palabras más frecuentes en cada clase
palabras_positivas = pd.DataFrame(X[df['airline_sentiment'] == 'positive'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)
palabras_negativas = pd.DataFrame(X[df['airline_sentiment'] == 'negative'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)

# Crear DataFrames para las palabras más frecuentes en cada clase
palabras_positivas_contador = pd.DataFrame({'Palabra': palabras_positivas.index, 'Conteo': palabras_positivas.values})
palabras_negativas_contador = pd.DataFrame({'Palabra': palabras_negativas.index, 'Conteo': palabras_negativas.values})

# Ordenar los DataFrames por conteo en orden descendente
palabras_positivas_contador = palabras_positivas_contador.sort_values(by='Conteo', ascending=False)
palabras_negativas_contador = palabras_negativas_contador.sort_values(by='Conteo', ascending=False)

# Imprimir las palabras más frecuentes en cada clase
print("Palabras más frecuentes en tweets positivos:")
print(palabras_positivas_contador.head(10))

print("\nPalabras más frecuentes en tweets negativos:")
print(palabras_negativas_contador.head(10))


Palabras más frecuentes en tweets positivos:
            Palabra  Conteo
13120           the     972
13326            to     938
14944           you     913
6157            for     670
13103        thanks     611
7804        jetblue     595
12372  southwestair     576
13914        united     528
13095         thank     455
2281            and     451

Palabras más frecuentes en tweets negativos:
         Palabra  Conteo
13326         to    6048
13120        the    4114
6004      flight    2943
13914     united    2899
2281         and    2825
9815          on    2792
14944        you    2722
6157         for    2714
9248          my    2407
14086  usairways    2375


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np


# Calcular el tercer cuartil de las frecuencias
tercer_cuartil_positivo = np.percentile(palabras_positivas_contador['Conteo'], 99.5)
tercer_cuartil_negativo = np.percentile(palabras_negativas_contador['Conteo'], 99.5)

# Identificar palabras comunes dentro del tercer cuartil
palabras_comunes_tercer_cuartil = set(palabras_positivas_contador[palabras_positivas_contador['Conteo'] >= tercer_cuartil_positivo]['Palabra']).intersection(
    set(palabras_negativas_contador[palabras_negativas_contador['Conteo'] >= tercer_cuartil_negativo]['Palabra']))

# Eliminar palabras comunes dentro del tercer cuartil
palabras_eliminar = [palabra for palabra in palabras_comunes_tercer_cuartil if
                     palabra in palabras_positivas_contador['Palabra'].values and
                     palabra in palabras_negativas_contador['Palabra'].values]

# Eliminar las palabras comunes
palabras_positivas_contador = palabras_positivas_contador[~palabras_positivas_contador['Palabra'].isin(palabras_eliminar)]
palabras_negativas_contador = palabras_negativas_contador[~palabras_negativas_contador['Palabra'].isin(palabras_eliminar)]

# Imprimir las palabras más frecuentes en cada clase después de eliminar las comunes
print("Palabras más frecuentes en tweets positivos:")
print(palabras_positivas_contador.head(10))

print("\nPalabras más frecuentes en tweets negativos:")
print(palabras_negativas_contador.head(10))

# Imprimir las palabras eliminadas
print("\nPalabras eliminadas en ambas clases dentro del tercer cuartil:")
print(palabras_eliminar)


Palabras más frecuentes en tweets positivos:
             Palabra  Conteo
13103         thanks     611
13095          thank     455
6656           great     236
14273  virginamerica     156
8532            love     136
6750            guys     110
6586            good     109
9209            much     109
2961            best     105
6610             got     100

Palabras más frecuentes en tweets negativos:
         Palabra  Conteo
3495   cancelled     926
2906        been     774
7158       hours     649
7051        hold     614
14551       what     588
10378      plane     532
14609        why     525
4666     delayed     508
12586      still     492
14570       when     473

Palabras eliminadas en ambas clases dentro del tercer cuartil:
['it', 'that', 'to', 'was', 'from', 'us', 'now', 'but', 'do', 'and', 'no', 'we', 'be', 'amp', 'out', 'with', 'time', 'is', 'me', 'for', 'at', 'not', 'so', 'this', 'our', 'all', 'gate', 'can', 'americanair', 'united', 'my', 'usairways', 'of', 'co', 'se